In [ ]:
# checking general null values, not treating specific erros for each column, just checking for null values or empty

In [ ]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# sparkSession = spark --> in the case of EMR
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

In [ ]:
from pyspark.sql.functions import col
import numpy as np

custom_schema = StructType([
    StructField("dt_server", StringType()),
    StructField("dt_avl", StringType()),
    StructField("line_id", IntegerType()),
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType()),
    StructField("id_avl", IntegerType()),
    StructField("event", IntegerType()),
    StructField("id_point", IntegerType())
])

In [ ]:
file_with_nulls = ["Nenhum"]
for day in range(1,32):
    filename = "MO_1510" + str(day)
    df = spark.read.csv("s3a://mobility-traces-sp/raw-data/"+ filename + ".csv",header="false",schema=custom_schema)
    x = [df.where((col(column).isNull()) | isnan(col(column)) | (col(column) == "")).limit(1).count() for column in df.columns]
    num_nulls = np.sum(x)
    if num_nulls > 0:
        file_with_nulls.append(filename)
    output = spark.createDataFrame([file_with_nulls])
    output.write.csv("s3://mobility-traces-sp/statistics/null-values/" + filename)